In [1]:
import torch
import time

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
SIZE = 4
for i in range(0, 4):
    cpu_start = time.time()
    cpu_arr = torch.ones(4 * (10 ** i), 4 * (10 ** i))
    for _ in range(1000):
        cpu_arr += cpu_arr
    print("CPU Time ({}): {}".format(i, time.time() - cpu_start))
    print(i)

0
1
2
3


In [11]:
start_time = time.time()
cpu_arr = torch.ones(4, 4)
for _ in range(1000):
    cpu_arr += cpu_arr
print("CPU time: ", time.time() - start_time)

CPU time:  0.0015006065368652344


In [12]:
start_time = time.time()
gpu_arr = torch.ones(4, 4).to(device)
for _ in range(1000):
    gpu_arr += gpu_arr
print("GPU time: ", time.time() - start_time)

GPU time:  0.004997968673706055
